In [96]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import tree, svm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

xl_file = '/Users/JaehoByun/JB/_School/2021_2 데이터사이언스/과제및시험/db_score_3_labels.xlsx'

db_score = pd.read_excel(xl_file)

In [37]:
db_score

,sno,homework,discussion,final,grade
0,1,19.37,0,30.10,A
1,2,20.00,0,30.10,A
2,3,19.37,0,29.75,A
3,4,18.12,2,29.05,A
4,5,20.00,2,21.35,A
...,...,...,...,...,...
87,88,20.00,0,2.80,C
88,89,19.37,0,4.55,C
89,90,18.75,0,0.00,C
90,91,0.00,0,1.05,C


In [39]:
X = db_score.loc[:,['homework', 'discussion', 'final']]
# X = [ (t['homework'], t['discussion'], t['final'] ) for t in db_score ]
X = np.array(X)
X

array([[19.37,  0.  , 30.1 ],
       [20.  ,  0.  , 30.1 ],
       [19.37,  0.  , 29.75],
       [18.12,  2.  , 29.05],
       [20.  ,  2.  , 21.35],
       [19.37,  0.  , 32.2 ],
       [19.37,  2.  , 28.7 ],
       [16.87,  0.  , 25.9 ],
       [19.37,  0.  , 21.7 ],
       [20.  ,  0.  , 19.25],
       [19.37,  0.  , 21.35],
       [19.37,  0.  , 20.65],
       [19.37,  0.  , 21.  ],
       [20.  ,  0.  , 18.55],
       [18.12,  0.  , 23.1 ],
       [19.37,  0.  , 24.85],
       [20.  ,  0.  , 18.55],
       [19.37,  0.  , 18.55],
       [19.37,  0.  , 24.5 ],
       [19.37,  0.  , 20.65],
       [19.37,  0.  , 12.95],
       [16.87,  0.  , 15.05],
       [19.37,  0.  , 17.5 ],
       [19.37,  0.  , 18.9 ],
       [19.37,  2.  , 10.15],
       [16.87,  0.  , 18.9 ],
       [19.37,  2.  , 22.05],
       [19.37,  0.  , 11.9 ],
       [16.25,  0.  , 22.4 ],
       [16.87,  0.  , 19.95],
       [15.  ,  0.  , 19.95],
       [20.  ,  0.  , 12.6 ],
       [19.37,  0.  , 19.95],
       [20

In [44]:
y = [ 1 if (t == 'B') else -1 for t in db_score['grade']]
y = np.array(y)

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1])

In [212]:
y = []
for t in db_score['grade']:
    if t == 'A':
        y.append(0)
    elif t == 'B':
        y.append(1)
    else:
        y.append(2)
    
y = np.array(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2])

In [173]:
def classification_performance_eval(y, y_predict):
    tp, tn, fp, fn = 0,0,0,0
    
    for y, yp in zip(y,y_predict):
        if y == 1 and yp == 1:
            tp += 1
        elif y == 1 and yp == -1:
            fn += 1
        elif y == -1 and yp == 1:
            fp += 1
        else:
            tn += 1

    print("tp={}, tn={}, fp={}, fn={}".format(tp,tn,fp,fn))

#     accuracy = (tp+tn)/(tp+tn+fp+fn)
#     precision = (tp)/(tp+fp)
#     recall = (tp)/(tp+fn)
#     f1_score = 2*precision*recall / (precision+recall)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    precision = (tp)/(tp+fp) if (tp+fp) != 0 else 0
    recall = (tp)/(tp+fn) if (tp+fn) != 0 else 0
    f1_score = 2*precision*recall / (precision+recall) if precision != 0 else 0
    
    return accuracy, precision, recall, f1_score

In [213]:
def multi_classification_performance_eval(y, y_predict):
    tp, tn, fp, fn = 0,0,0,0
    
    for y, yp in zip(y,y_predict):
        if y == 0 and yp == 0 or y == 1 and yp == 1 or y == 2 and yp == 2:
            tp += 1
        elif y == 0 and yp != 0 or y == 1 and yp != 1 or y == 2 and yp != 2:
            fn += 1
        elif y != 0 and yp == 0 or y != 1 and yp == 1 or y != 2 and yp == 2:
            fp += 1
        elif y != 0 and yp != 0 or y != 1 and yp != 1 or y != 2 and yp != 2:
            tn += 1

    print("tp={}, tn={}, fp={}, fn={}".format(tp,tn,fp,fn))

#     accuracy = (tp+tn)/(tp+tn+fp+fn)
#     precision = (tp)/(tp+fp)
#     recall = (tp)/(tp+fn)
#     f1_score = 2*precision*recall / (precision+recall)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    precision = (tp)/(tp+fp) if (tp+fp) != 0 else 0
    recall = (tp)/(tp+fn) if (tp+fn) != 0 else 0
    f1_score = 2*precision*recall / (precision+recall) if precision != 0 else 0
    
    return accuracy, precision, recall, f1_score

In [219]:
# [Algorithm 1] Decision Tree model classification
def decision_tree_model(X_train, X_test, y_train, y_test):
    classifier = tree.DecisionTreeClassifier()
    dtree_model = classifier.fit(X_train, y_train)

    y_predict = dtree_model.predict(X_test)
    print(y_test)
    print(y_predict)

    print(classification_report(y_test, y_predict))
    print(accuracy_score(y_test, y_predict))
    print(precision_score(y_test, y_predict))
    print(recall_score(y_test, y_predict))
    print(f1_score(y_test, y_predict))
#     acc, prec, rec, f1 = classification_performance_eval(y_test, y_predict)

#     return acc, prec, rec, f1

In [209]:
# [Algorithm 2] SVM(Support Vector Machine) model classification
def svm_model(X_train, X_test, y_train, y_test):
    classifier = svm.SVC(kernel='rbf', C=8, gamma=0.1)  # 여기를 조절해야댐
    svm_model = classifier.fit(X_train, y_train)

    y_predict = svm_model.predict(X_test)
    print(y_test)
    print(y_predict)
    
    classification_report(y_test, y_predict)
    print(accuracy_score(y_test, y_predict))
    print(precision_score(y_test, y_predict))
    print(recall_score(y_test, y_predict))
    print(f1_score(y_test, y_predict))
    
#     acc, prec, rec, f1 = classification_performance_eval(y_test, y_predict)

#     return acc, prec, rec, f1

In [220]:
# [Algorithm 3] Logistic Regression model classification
def logistic_regression_model(X_train, X_test, y_train, y_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    classifier = LogisticRegression()
    lg_model = classifier.fit(X_train_scaled, y_train)

    y_predict = lg_model.predict(X_test_scaled)
    print(y_test)
    print(y_predict)
    
    classification_report(y_test, y_predict)
    print(accuracy_score(y_test, y_predict))
    print(precision_score(y_test, y_predict))
    print(recall_score(y_test, y_predict))
    print(f1_score(y_test, y_predict))

#     acc, prec, rec, f1 = classification_performance_eval(y_test, y_predict)

#     return acc, prec, rec, f1

In [191]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42)

In [214]:
# decision_tree_model(X_train, X_test, y_train, y_test)

acc, prec, rec, f1 = decision_tree_model(X_train, X_test, y_train, y_test)

print("< Decision Tree >")
print("accuracy=%f" %acc)
print("precision=%f" %prec)
print("recall=%f" %rec)
print("f1_score=%f" %f1)
print("")

[ 1 -1  1 -1 -1 -1  1 -1 -1  1 -1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1  1
  1 -1 -1 -1  1 -1 -1]
[-1  1  1 -1 -1 -1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1  1
  1 -1  1 -1 -1 -1 -1]
tp=4, tn=18, fp=4, fn=5
< Decision Tree >
accuracy=0.709677
precision=0.500000
recall=0.444444
f1_score=0.470588



In [210]:
# svm_model(X_train, X_test, y_train, y_test)

acc, prec, rec, f1 = svm_model(X_train, X_test, y_train, y_test)

print("< SVM (Support Vector Machine) >")
print("accuracy=%f" %acc)
print("precision=%f" %prec)
print("recall=%f" %rec)
print("f1_score=%f" %f1)
print("")

[ 1 -1  1 -1 -1 -1  1 -1 -1  1 -1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1  1
  1 -1 -1 -1  1 -1 -1]
[ 1  1  1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1  1  1
  1 -1  1 -1 -1  1 -1]
tp=5, tn=17, fp=5, fn=4
< SVM (Support Vector Machine) >
accuracy=0.709677
precision=0.500000
recall=0.555556
f1_score=0.526316



In [197]:
# logistic_regression_model(X_train, X_test, y_train, y_test)

acc, prec, rec, f1 = logistic_regression_model(X_train, X_test, y_train, y_test)

print("< Logistic Regression >")
print("accuracy=%f" %acc)
print("precision=%f" %prec)
print("recall=%f" %rec)
print("f1_score=%f" %f1)
print("")

[ 1 -1  1 -1 -1 -1  1 -1 -1  1 -1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1  1
  1 -1 -1 -1  1 -1 -1]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1]
tp=0, tn=22, fp=0, fn=9
< Logistic Regression >
accuracy=0.709677
precision=0.000000
recall=0.000000
f1_score=0.000000



In [215]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42)

In [221]:
# # decision_tree_model(X_train, X_test, y_train, y_test)

decision_tree_model(X_train, X_test, y_train, y_test)
# acc, prec, rec, f1 = decision_tree_model(X_train, X_test, y_train, y_test)

# print("< Decision Tree >")
# print("accuracy=%f" %acc)
# print("precision=%f" %prec)
# print("recall=%f" %rec)
# print("f1_score=%f" %f1)
# print("")

# # svm_model(X_train, X_test, y_train, y_test)

svm_model(X_train, X_test, y_train, y_test)
# acc, prec, rec, f1 = svm_model(X_train, X_test, y_train, y_test)

# print("< SVM (Support Vector Machine) >")
# print("accuracy=%f" %acc)
# print("precision=%f" %prec)
# print("recall=%f" %rec)
# print("f1_score=%f" %f1)
# print("")

# # logistic_regression_model(X_train, X_test, y_train, y_test)

logistic_regression_model(X_train, X_test, y_train, y_test)
# acc, prec, rec, f1 = logistic_regression_model(X_train, X_test, y_train, y_test)

# print("< Logistic Regression >")
# print("accuracy=%f" %acc)
# print("precision=%f" %prec)
# print("recall=%f" %rec)
# print("f1_score=%f" %f1)
# print("")

[1 0 1 2 0 0 1 2 0 1 2 1 2 2 0 0 0 0 1 2 0 2 2 1 1 0 2 2 1 2 0]
[0 1 1 2 0 0 1 1 0 1 2 0 2 2 0 0 0 0 2 0 0 0 2 1 1 0 1 2 0 2 0]
              precision    recall  f1-score   support

           0       0.67      0.91      0.77        11
           1       0.62      0.56      0.59         9
           2       0.88      0.64      0.74        11

    accuracy                           0.71        31
   macro avg       0.72      0.70      0.70        31
weighted avg       0.73      0.71      0.71        31

0.7096774193548387


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].